# Dining Philosophers
Daniel Rothfusz

In [1]:
import threading
import time
import random
from pynq.overlays.base import BaseOverlay
import pynq.lib.rgbled as rgbled
base = BaseOverlay("base.bit")

In [2]:
def blink(t, p, n, sign=None):
    '''
    Function to blink the LEDs
    Params:
      t: time (in seconds) for the blink pattern to continue
      p: period of a full on-off cycle of blinking (in seconds)
      n: index of the LED (0 to 4) with index 4 being an RGB LED
      sign: threading Event, if present check if set to abort function early
    '''
    g_led=rgbled.RGBLED(4)
    for iteration in range (int(2*t/p)):
        if n==4:
            if g_led.read () > 0:
                g_led.off ()
            else:
                g_led.write (0x2)
        else:
            
            base.leds[n].toggle()
        time.sleep(p/2)
        if (sign != None):
            if sign.is_set ():
                break
    # Reset LEDS
    if n==4:
        g_led.off ()
    else:
        base.leds[n].off()

### Test Blink Function

In [3]:
for i in range (5):
    blink(5, 1, i)


### Deterministic Timing

In [35]:
eating_duration=3
eating_period=0.2
napping_duration=1
napping_period=1

btns = base.btns_gpio

signal = threading.Event ()

def philosopher(fork1_, fork2_, num):
    global signal
    while not signal.is_set():
        fork1_.acquire(True)
        fork2_.acquire(True)
        blink(eating_duration, eating_period, num, signal)
        fork2_.release()
        fork1_.release()
        if signal.is_set ():
            continue   
        blink(napping_duration, napping_period, num, signal)
        
        
def buttons():
    global signal
    while not signal.is_set():
        time.sleep(0.1)
        if btns.read() != 0:
            print(f"Stop signal sent")
            signal.set()

forks=[]
for i in range(5):
    forks.append(threading.Lock())
threads=[]
for i in range(5):
    t = threading.Thread(target=philosopher, args=[forks[i], forks[(i+1)%5], i])
    threads.append(t)
    t.start()
btn = threading.Thread(target=buttons, args=[])
threads.append(btn)
btn.start()

for t in threads:
    name = t.name
    t.join()
    print('{} joined'.format(name))

Stop signal sent
Thread-89 (philosopher) joined
Thread-90 (philosopher) joined
Thread-91 (philosopher) joined
Thread-92 (philosopher) joined
Thread-93 (philosopher) joined
Thread-94 (buttons) joined


### Random Timing

In [7]:
base_eating_duration=3
eating_period=0.2
base_napping_duration=1
napping_period=1

btns = base.btns_gpio

signal = threading.Event ()

def philosopher(fork1_, fork2_, num):
    global signal
    while not signal.is_set():
        fork1_.acquire(True)
        fork2_.acquire(True)
        blink(base_eating_duration+random.randint(0,2), eating_period, num, signal)
        fork2_.release()
        fork1_.release()
        if signal.is_set ():
            continue   
        blink(base_napping_duration+random.randint(0,2), napping_period, num, signal)
        
        
def buttons():
    global signal
    while not signal.is_set():
        time.sleep(0.1)
        if btns.read() != 0:
            print(f"Stop signal sent")
            signal.set()

forks=[]
for i in range(5):
    forks.append(threading.Lock())
threads=[]
for i in range(4):
    t = threading.Thread(target=philosopher, args=[forks[i], forks[(i+1)%5], i])
    threads.append(t)
    t.start()
# Last thread has alternate order of fork pickup to prevent deadlock/starvation cycles
last_thread=threading.Thread(target=philosopher, args=[forks[0], forks[4], 4])
threads.append(last_thread)
last_thread.start()
btn = threading.Thread(target=buttons, args=[])
threads.append(btn)
btn.start()

for t in threads:
    name = t.name
    t.join()
    print('{} joined'.format(name))

Stop signal sent
Thread-23 (philosopher) joined
Thread-24 (philosopher) joined
Thread-25 (philosopher) joined
Thread-26 (philosopher) joined
Thread-27 (philosopher) joined
Thread-28 (buttons) joined
